<a href="https://colab.research.google.com/github/YuriyKozhubaev/PY100/blob/main/PY110_lecture_2_1_Generator_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Функции-генераторы

![common iterators description](https://github.com/aeksei/py110-lecture/blob/master/images/generators.png?raw=true)

## Выражения генераторы 
Быстро освежим в памяти как работают comprehensions

In [ ]:
a = [i for i in range(1, 12, 3)]  
b = {i for i in "hello, world"}
c = {i: i.upper() for i in "abcdefghijklm"}

In [ ]:
print(a)
print(b)
print(c)

[1, 4, 7, 10]
{'w', 'e', 'o', ' ', 'r', ',', 'h', 'l', 'd'}
{'a': 'A', 'b': 'B', 'c': 'C', 'd': 'D', 'e': 'E', 'f': 'F', 'g': 'G', 'h': 'H', 'i': 'I', 'j': 'J', 'k': 'K', 'l': 'L', 'm': 'M'}


In [ ]:
# Примеры чуть посложнее:
my_list = list(range(10))
x = [i**3 for i in my_list if i > 4 and i % 2 == 1]
y = {f"{x} * {y}": x * y for x in range(10) for y in range(7, 0 ,-1) if x * y > 10}

In [ ]:
print(x)
print(y)

[125, 343, 729]
{'2 * 7': 14, '2 * 6': 12, '3 * 7': 21, '3 * 6': 18, '3 * 5': 15, '3 * 4': 12, '4 * 7': 28, '4 * 6': 24, '4 * 5': 20, '4 * 4': 16, '4 * 3': 12, '5 * 7': 35, '5 * 6': 30, '5 * 5': 25, '5 * 4': 20, '5 * 3': 15, '6 * 7': 42, '6 * 6': 36, '6 * 5': 30, '6 * 4': 24, '6 * 3': 18, '6 * 2': 12, '7 * 7': 49, '7 * 6': 42, '7 * 5': 35, '7 * 4': 28, '7 * 3': 21, '7 * 2': 14, '8 * 7': 56, '8 * 6': 48, '8 * 5': 40, '8 * 4': 32, '8 * 3': 24, '8 * 2': 16, '9 * 7': 63, '9 * 6': 54, '9 * 5': 45, '9 * 4': 36, '9 * 3': 27, '9 * 2': 18}


---
В чем минус comprehensions?

Он создает сразу объект целиком, который целиком в памяти хранится.  
Это полезно для словарей и множеств, а также для списков,  
с которыми мы потом будем много раз работать.

Но что, если нам нужен список, по которому мы будем перечислять и  
он нужен только один раз?  
А если мы его хотим бесконечной длины??
```python
a = [i**2 for i in itertools.count(1, 1)]
```

И тут нам на помощь приходят генераторы.  
Они отличаются от вложений тем, что не вычисляются сразу,  
а только в тот момент, когда мы обращаемся к следующему элементу

Отличие от comprehensions только в том, что мы используем круглые скобки
``` python
a = (i**2 for i in itertools.count(1, 1))
```

Данное выражение возвращает итерируемый объект,  
который сам по себе является итератором.  
Т.е. можно сразу его перечислять (с помощью `for` или `next`),  
а можно взять `iter(a)` и применять `next` к итератору (но не имеет смысла).

In [ ]:
sqrt_gen_exp = (i ** 2 for i in range(1, 11))  # выражение генератор
print(type(sqrt_gen_exp))

<class 'generator'>


In [ ]:
print(next(sqrt_gen_exp))
print(next(sqrt_gen_exp))

1
4


In [ ]:
iter(sqrt_gen_exp) 

<generator object <genexpr> at 0x7f38d7fcbb50>

In [ ]:
sqrt_gen_exp is iter(sqrt_gen_exp) 

True

**Основной плюс генераторов** – они не хранят все элементы в памяти,  
а вычисляют очередной элемент на момент обращения к нему.

**Основной минус** – с такой записью генераторов достаточно сложно  
реализовать сложную логику подсчета следующего элемента 
(хочу генератор чисел Фибоначчи, например)

А с какой записью легче? Увидим дальше. 

## Функция-генератор

Выражения-генераторы позволяют нам создавать генераторы,  
не хранящие элементы в памяти, а вычисляющие их при каждом запросе.  
Однако, у выражений есть минус – с помощью них нельзя реализовать  
сложную логику.  

Для сложного существуют **функции-генераторы**.

```python
def gen_function(*args, **kwargs):
    # тело генератора
    
    yield result
    
    # конец тела генератора
```

**Функции-генераторы** – обычные функции, в которых вместо слова `return`  
используется ключевое слово `yield`.  
При этом при вызове функции вы получаете объект-генератор,  
реализующий внутреннюю логику функции.  

Каждый раз, когда выполняется `yield`, объект сохраняет свой контекст  
выполнения (значения всех переменных, стек вызовов и т.д.)  
При следующем вызове `next` к этому объекту контекст восстанавливается  
и объект продолжает выполняться с этой же точки  

In [ ]:
# попробуем реализовать бесконечный счетчик count из модуля itertools
def count(start=1, step=1):
    counter = start
    while True:
        yield counter
        counter += step
    return None

In [ ]:
my_gen_func = count(100, 10)
for _ in range(10):
    print(next(my_gen_func))  # с функцией генератором также работает next

100
110
120
130
140
150
160
170
180
190


In [ ]:
my_gen_func is iter(my_gen_func)

True

In [ ]:
next(my_gen_func)

200

---
Как и все генераторы, функция-генератор останавливается с вызовом `StopIteration`.  

Это происходит при следующих условиях:
1. Интерпретатор достиг конца выполнения функции и не встретил никаких инструкций
2. В функции был выполнен return

In [ ]:
def first_gen(input_: int):
    yield input_
    input_ += 1
    print(input_)
    # return None
    
my_first_gen = first_gen(5)

print(next(my_first_gen))

5


In [ ]:
# StopIteration
next(my_first_gen)  # print из тела генератор

6


StopIteration: ignored

In [ ]:
def second_gen(input_):
    yield input_
    input_ += 1
    
    yield input_
    input_ += 1
    
    return input_
    
    
my_second_gen = second_gen(10)

print(next(my_second_gen))
print(next(my_second_gen))
print(next(my_second_gen))

10
11


StopIteration: ignored

---
Разберем ещё один пример:

In [ ]:
def my_animal_generator():
    yield 'корова'
    print('---')
    for animal in ['кот', 'собака', 'медведь']:
        yield animal
    print('---')
    yield 'кит'

a = my_animal_generator()
print(next(a))
# print('---') вызван не будет

корова


In [ ]:
print(next(a))

for i in a:
    print(i)

---
кот
собака
медведь
---
кит


In [ ]:
for i in my_animal_generator():
    print(i)

корова
---
кот
собака
медведь
---
кит


---
Попробуем реализивать генератор чисел Фибоначчи.  
Последовательность чисел Фибоначчи представляет собой:  
0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711 ...

А общая формула имеет следующий вид:  

$$F_0=0,  F_1=1$$  

$$F_n=F_{n-1}+F_{n-2}, n\geq\ 2$$

In [ ]:
def fib():
    a, b = 0, 1
    yield a  # F0
    yield b  # F1

    while True:
        a, b = b, a + b
        yield b

In [ ]:
fib_gen = fib()
for num in fib_gen:
    print(num)
    if num > 10000:
        break

0
1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765
10946


In [ ]:
next(fib_gen)

17711

С помощью метода `close()` можно закрыть генератор  

In [ ]:
fib_gen = fib()
fib_gen.close()  # закрыли генератор

for i in fib_gen:  # цикл не выполнится ни одного раза
    print(i)
    
fib_gen

<generator object fib at 0x7f38d3c52050>

In [ ]:
next(fib_gen)

StopIteration: ignored

Итак, с помощью функции-генератора мы с вами написали эффективную реализацию последовательности чисел Фибоначчи, которой для вычисления необходимо хранить в памяти только два предыдущих числа, а вычисления происходят только в момент вызова функции-генератора. 

Функции генераторы применяются в тех случаях, когда вам:
- необходимо сэкономить память при работе с массивными структурами (ленивые вычисления)
- удобна логика работы генератора, внутри которого можно описать сложный процесс выбора

Пример, который вам может пригодиться – построчное чтение 5Гб файла  

Примеры модулей:
- `keras` – генератор данных для обучения
- `faker` – генерация фейковых данных для заполнения и тестирования объектов
- `gmp2` – генерация простых чисел




---
## Корутины (Coroutines)

В языке Python генераторы могут использоваться и в «обратную сторону» – принимая в себя внутрь значения.  
До этого мы только получали значения из наших генераторов, посмотрим теперь, как сделать так, чтобы можно было передавать информацию внутрь нашего генератора.

Попробуем сделать, корутину, которая будет принимать некий паттерн, проверять его наличие в какой-либо строке и подсчитывать количество вхождений.

In [ ]:
def grep(pattern):
    i = 0
    while True:
        line = yield  # отличие от генератора
        if pattern in line:
            i += 1
            print(f"Found! Count: {i}")

Отличие в синтаксисе только в том, что теперь слева от `yield` стоит переменная, которая будет принимать значение, которое мы передаем из вне.

In [ ]:
g = grep("python")  # создаем объект корутину, которой в качетве аргумента передаем паттерн

Единственная особенность в том, чтобы запустить корутину, её нужно проинициализировать.
Делается это с помощью команды `next(coroutine)` либо `coroutine.send(None)`, что является одним и тем же

$$next(coroutine) \equiv coroutine.send(None)$$

Ранее, когда мы делали команду `next()` к нашему генератору, мы делали ему `send(None)`

In [ ]:
# инициализируем корутину
g.send(None)  # либо next(g)

In [ ]:
# отправим нашей корутине, новое значение для проверки вхождения в строку
g.send("some string here python")

Found! Count: 1


In [ ]:
g.send("anouther string")

In [ ]:
g.send("python python!!!")

Found! Count: 2


In [ ]:
# чтобы закрыть корутину можно использовать метод close
g.close()

Попробуем прислать нашей корутине, первым действием что-то отличное от `None`

In [ ]:
g = grep("new_pattern")

g.send("New string with new_pattern")

TypeError: ignored

И получим соответсвующую ошибку, что только что стартовавшему генератору, нельзя присылать значение отличное от `None`

---
Рассмотрим момент, который окончательно "вынесет мозг". Это когда корутина может одновременно принимать и отправлять данные.  

Давайте сделаем генератор, который будет возвращать нам бесконечную арифметическую прогрессию, то мы сможем переопределять начальные значения после запуска генератора.

In [ ]:
def counter(start, step):
    n = start
    while True:
        input_data = yield n
        if input_data is not None:
            n = input_data[0]
            step = input_data[1]
        else:
            n += step

In [ ]:
c = counter(100, 10)  # создаем корутину с начальными значениями
for _ in range(10):
    print(c.send(None))

In [ ]:
# передаем корутине новые значения старта и шага прогресии без создания новой корутины
print(c.send((500, 1)))  

In [ ]:
for _ in range(10):
    print(c.send(None))
    
c.close()

---
### Пример использования:
Живой пример использования данной идеи – модуль tornado
Асинхронная сетевая библиотека для обработки запросов
скалируется до десятков тысяч одновременно открытых соединений
построен на генераторах и корутинах (с кучей дополнительного, конечно же)

Разница между Django и Tornado на задачах вида «загрузи ответ от другого сервера и верни мне данные» – до 10 раз в пользу торнадо (и других асинхронных фреймворков) [источник](https://klen.github.io/python-web-benchmarks.html)

---
### Подведем итоги по генераторам и корутинам:
В более общем смысле, генераторы и корутины – это объекты, которые умеют получать управление, выполнять некоторую работу и отдавать управление обратно.  
Данная логика работы называется «кооперативная многозадачность». Грамотно реализуя преимущества данной логики, можно построить на языке Python приложения, обрабатывающие десятки тысяч запросов ежесекундно.

Более подробно с многозачачностями можно познакомиться [тут](https://kvckr.github.io/mag/sp/7.html)